In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#default_exp download

# Download

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import shutil
from numerapi import NumerAPI, SignalsAPI
from pathlib import Path, PosixPath
from abc import ABC
from rich import print as rich_print

### 1. Base

`BaseDownloader` is a simple object which implements logic common to all downloaders.

To implement a new Downloader, you should inherit from `BaseDownloader` and be sure to implement at least `download_training_data` and `download_inference_data`.

In [ ]:
#export
class BaseDownloader(ABC):
    """
    Abstract base class for downloaders.
    :param directory_path: Base directory where data will be saved.
    """
    def __init__(self, directory_path: str):
        self.dir = Path(directory_path)
        if not self.dir.is_dir():
            rich_print(f"No existing directory found at '[blue]{self.dir}[/blue]'. Creating directory...")
            self.dir.mkdir(parents=True, exist_ok=True)

    def download_training_data(self, *args, **kwargs):
        raise NotImplementedError(f"No method for downloading training data is implemented in '{self.__class__.__name__}'")

    def download_inference_data(self, *args, **kwargs):
        raise NotImplementedError(f"No method for downloading inference data is implemented in '{self.__class__.__name__}'.")

    def remove_directory(self):
        """ Remove download directory with all contents. """
        abs_path = self.dir.resolve()
        rich_print(f":warning: [red]Deleting directory for '{self.__class__.__name__}[/red]': :warning:\nPath: '{abs_path}'")
        shutil.rmtree(abs_path)

    @property
    def get_all_files(self):
        """ Return all contents in directory. """
        return list(base_down.dir.iterdir())

    @property
    def is_empty(self):
        """ Check if directory is empty."""
        return not bool(self.get_all_files)

    def __call__(self, *args, **kwargs):
        """
        The most common use case will be to get weekly inference data. So calling the class itself returns inference data.
        """
        self.download_inference_data(*args, **kwargs)

In [ ]:
test_dir = "test_base_1234321234321/"

# Test building class
base_down = BaseDownloader(directory_path=test_dir)
assert isinstance(base_down.dir, PosixPath)
assert base_down.dir.is_dir()

# Test properties
(base_down.dir / "test.txt").write_text("test")
rich_print(f"Directory contents:\n{base_down.get_all_files}")
assert not base_down.is_empty

# Remove contents
base_down.remove_directory()
assert not os.path.exists(test_dir)

No existing directory found at 'test_base_1234321234321'. Creating directory...

Directory contents:
[PosixPath('test_base_1234321234321/test.txt')]

⚠ Deleting directory for 'BaseDownloader': ⚠
Path: 
'/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_base_1234321234321'

## 2. Numerai Classic

In [ ]:
class NumeraiClassicDownloader(BaseDownloader):
    def __init__(self, directory_path: str, version: int = 2, *args, **kwargs):
        super(NumeraiClassicDownloader, self).__init__(directory_path=directory_path)
        self.api = NumerAPI(*args, **kwargs)
        self.version = version

    def download_training_data(self):
        # TODO Implement train set downloading
        ...

    def download_inference_data(self):
        # TODO implement inference downloading
        ...

In [ ]:
test_dir_classic = "test_numerai_classic_1234321"
numer_classic_downloader = NumeraiClassicDownloader(test_dir_classic)

# TODO test for ClassicDownloader (including full #slow tests)

# Remove contents
numer_classic_downloader.remove_directory()
assert not os.path.exists(test_dir_classic)

⚠ Deleting directory for 'NumeraiClassicDownloader': ⚠
Path: '/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_numerai_class
ic_1234321'

## 3. Numerai Signals